# Colab 4: GRPO Reinforcement Learning (Reasoning) — Simple
This notebook fine-tunes a small model with **GRPO** on GSM8K problems.
The model generates answers; a simple reward checks correctness; GRPO updates the policy.


In [1]:
!pip install -q unsloth transformers datasets accelerate bitsandbytes peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [2]:
import os, re
os.environ['WANDB_DISABLED'] = 'true'
import torch
torch.backends.cuda.matmul.allow_tf32 = True
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import GRPOTrainer, GRPOConfig
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

Device: cuda


## Load reasoning problems (GSM8K)

In [3]:
raw = load_dataset('openai/gsm8k', 'main')
train = raw['train'].select(range(600))
print('Samples:', len(train))

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Samples: 600


## Build prompts with IDs and collect gold answers

In [4]:
def make_prompt(q, i):
    return (
        f'ID={i}\n'
        f'Question: {q}\n'
        "Think step by step and provide the final answer as '#### <number>'.\n"
        'Answer:'
    )
questions = [ex['question'] for ex in train]
gold      = [ex['answer']   for ex in train]
prompts   = [make_prompt(q, i) for i, q in enumerate(questions)]
train_ds_for_grpo = [{'prompt': p} for p in prompts]
print(train_ds_for_grpo[0]['prompt'][:200])

ID=0
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Think step by step and provid


## Load small model and attach LoRA

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/smollm2-135m',
    dtype=torch.float16,
    load_in_4bit=True,
)
model.gradient_checkpointing_enable()
model.config.use_cache = False
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias='none',
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
)
print('Model ready.')

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.2 patched 30 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Model ready.


## Reward function: +1 if final number matches gold, else 0

In [9]:

id_re  = re.compile(r"^ID=(\d+)", re.M)
num_re = re.compile(r"####\s*(-?\d+)")

def to_text(x):
    if isinstance(x, str):
        return x
    if isinstance(x, dict):
        for k in ("prompt","text","generated_text","content","response","answer"):
            if k in x and isinstance(x[k], str):
                return x[k]
        if "messages" in x and isinstance(x["messages"], list):
            return "\n".join(m.get("content","") if isinstance(m, dict) else str(m) for m in x["messages"])
        return str(x)
    if isinstance(x, list):
        return "\n".join(to_text(y) for y in x)
    return str(x)

# NOTE: signature uses `prompts` (not prompts_batch) to match your GRPO trainer
def reward_funcs(completions, prompts, completion_ids=None, **kwargs):
    rewards = []
    for comp, pr in zip(completions, prompts):
        comp_text = to_text(comp)
        pr_text   = to_text(pr)

        m_id   = id_re.search(pr_text)         # recover the ID we embedded in the prompt
        m_pred = num_re.search(comp_text)      # model's final "#### <number>"
        if not m_id or not m_pred:
            rewards.append(0.0)
            continue

        idx = int(m_id.group(1))
        m_gold = num_re.search(gold[idx]) if 0 <= idx < len(gold) else None  # gold comes from GSM8K list you built earlier
        rewards.append(1.0 if (m_gold and m_pred.group(1) == m_gold.group(1)) else 0.0)
    return rewards


## Configure GRPO (minimal)

In [10]:
config = GRPOConfig(
    output_dir='./smollm2_grpo_gsm8k',
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-6,
    num_train_epochs=1,
    max_prompt_length=256,
    max_completion_length=128,
    num_generations=2,
)
if not hasattr(config, 'model_init_kwargs'):
    config.model_init_kwargs = {}
print(config)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


UnslothGRPOConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
beta=0.001,
bf16=False,
bf16_full_eval=False,
cache_implementation=None,
data_seed=3407,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
delta=None,
disable_dropout=False,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
ds3_gather_for_generation=True,
epsilon=0.2,
epsilon_high=None,
eval_accumulation_steps=2,
eval_delay=0

## Trainer: model generates answers; reward guides updates

In [11]:
trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer,
    args=config,
    train_dataset=train_ds_for_grpo,
    reward_funcs=reward_funcs,
)
print('Starting GRPO training...')
trainer.train()
print('Training done.')

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting GRPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 2,442,240 of 136,957,824 (1.78% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / reward_funcs / mean,rewards / reward_funcs / std
1,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000024,0.000000,0.000000
3,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000032,0.000000,0.000000
4,0.000000,0.000000,0.000000,82.500000,35.000000,128.000000,0.500000,37.000000,35.000000,39.000000,No Log,No Log,No Log,No Log,No Log,0.000201,0.000000,0.000000
5,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000027,0.000000,0.000000
6,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000029,0.000000,0.000000
7,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000031,0.000000,0.000000
8,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000026,0.000000,0.000000
9,0.000000,0.000000,0.000000,128.000000,128.000000,128.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000027,0.000000,0.000000
10,0.000000,0.000000,0.000000,98.750000,11.000000,128.000000,0.750000,11.000000,11.000000,11.000000,No Log,No Log,No Log,No Log,No Log,0.000251,0.000000,0.000000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Training done.


## Save the fine-tuned reasoning model

In [12]:
model.save_pretrained('./smollm2_grpo_gsm8k')
tokenizer.save_pretrained('./smollm2_grpo_gsm8k')
print('Saved to ./smollm2_grpo_gsm8k')

Saved to ./smollm2_grpo_gsm8k


## Quick test generation on a new problem

In [13]:
test_q = 'A car travels 54 km at 18 km/h. How many hours did it travel? Answer as \'#### <number>\'.'
test_prompt = (
    'ID=99999\n'
    f'Question: {test_q}\n'
    "Think step by step and provide the final answer as '#### <number>'.\n"
    'Answer:'
)
inputs = tokenizer(test_prompt, return_tensors='pt').to(device)
out = model.generate(**inputs, max_new_tokens=96)
print(tokenizer.decode(out[0], skip_special_tokens=True))

ID=99999
Question: A car travels 54 km at 18 km/h. How many hours did it travel? Answer as '#### <number>'.
Think step by step and provide the final answer as '#### <number>'.
Answer: 18 hours.

Question: A car travels 54 km at 18 km/h. How many hours did it travel? Answer as '#### <number>'.
Think step by step and provide the final answer as '#### <number>'.
Answer: 18 hours.

Question: A car travels 54 km at 18 km/h. How many hours did it travel? Answer as '#### <number
